<a href="https://colab.research.google.com/github/unverciftci/agents_intro/blob/main/Notebook_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Tutorial 2: AI with Tools
# Goal: Transform chatbot into tool-using agent

import re
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load AI (same as Tutorial 1)
print("Loading AI...")
model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
print("✅ AI ready!")

# Step 1: Create tools
def calculator(expression):
    """Simple calculator tool"""
    try:
        result = eval(expression)
        return f"Result: {result}"
    except:
        return "Error: Invalid calculation"

def get_weather(city):
    """Weather tool (simulated data)"""
    weather_data = {
        "paris": "Sunny, 22°C",
        "london": "Rainy, 15°C",
        "tokyo": "Cloudy, 18°C",
        "new york": "Windy, 20°C"
    }
    return weather_data.get(city.lower(), "Weather data not available")

# Tool registry
TOOLS = {
    "calculator": calculator,
    "get_weather": get_weather
}

print("✅ Tools ready: calculator, get_weather")

# Step 2: Basic AI function
def ask_ai(prompt):
    """Send prompt to AI and get response"""
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True, enable_thinking=False
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=120,
            temperature=0.3,
            do_sample=True,
        )

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist()
    return tokenizer.decode(output_ids, skip_special_tokens=True).strip()

# Step 3: Agent with tools
def agent_with_tools(user_message):
    """AI agent that can use tools"""

    # System prompt with examples
    system_prompt = f"""You are an AI assistant that can use tools. When you need to:
- Calculate something: use TOOL:calculator:expression
- Get weather: use TOOL:get_weather:city

Examples:
User: What's 15 + 27?
AI: I'll calculate that for you. TOOL:calculator:15+27

User: What's the weather in Paris?
AI: Let me check the weather. TOOL:get_weather:paris

User: Hi there!
AI: Hello! How can I help you today?

User: {user_message}
AI:"""

    print(f"👤 User: {user_message}")

    # Get AI response
    ai_response = ask_ai(system_prompt)
    print(f"🤖 AI: {ai_response}")

    # Check if AI wants to use a tool
    if "TOOL:" in ai_response:
        tool_match = re.search(r'TOOL:(\w+):(.+)', ai_response)
        if tool_match:
            tool_name = tool_match.group(1)
            tool_input = tool_match.group(2).strip()

            print(f"🛠️ Using tool: {tool_name}({tool_input})")

            if tool_name in TOOLS:
                tool_result = TOOLS[tool_name](tool_input)
                print(f"✅ Tool result: {tool_result}")

                # Get final response with tool result
                final_prompt = system_prompt + ai_response + f"\nTool result: {tool_result}\nFinal response:"
                final_response = ask_ai(final_prompt)

                return f"{ai_response}\nTool result: {tool_result}\n{final_response}"
            else:
                return f"{ai_response}\nError: Unknown tool '{tool_name}'"

    return ai_response

# Step 4: Test the agent
print("\n🧪 Testing agent with tools:")

test_questions = [
    "What's 25 * 4?",
    "What's the weather in London?",
    "Hello! How are you?",
    "Calculate 100 / 5 please"
]

for question in test_questions:
    print(f"\n--- Test ---")
    response = agent_with_tools(question)
    print(f"Final response: {response}")
    print("-" * 40)

# Step 5: Interactive mode
def chat_with_agent():
    """Interactive chat with tool-using agent"""
    print("\n🛠️ Chat with your agent (now with tools!)")
    print("Try asking for calculations or weather!")
    print("Type 'quit' to exit")

    while True:
        user_input = input("\n👤 You: ")

        if user_input.lower() in ['quit', 'exit', 'q']:
            print("👋 Goodbye!")
            break

        if user_input.strip():
            response = agent_with_tools(user_input)
            print(f"🤖 Final: {response}")

# Ready to test? Uncomment the line below:
# chat_with_agent()

print("\n🎉 Your AI can now use tools!")
print("🚀 Next: Tutorial 3 - Add memory")

KeyboardInterrupt: 